In [1]:
import requests
from bs4 import BeautifulSoup 
import os
import pandas as pd

In [2]:
final_stats_list = []
flag = "a"

In [3]:
def download(filename, url):
    if os.path.exists(filename):
        return (str(filename) + " already exists!")

    r = requests.get(url)
    r.raise_for_status()
    
    f = open(filename, "w", encoding="utf-8")
    f.write(r.text)
    f.close()

    return (str(filename) + " created!")

In [4]:
def scrape_from_web(model_name):
    filename = "_".join(model_name.split()) + ".html"
    url = "https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2380057.m570.l1313&_nkw=" + \
        "+".join(model_name.split()) + "&_sacat=0"
    download(filename, url)
    return filename

In [5]:
def parse_HTML(filename):
    file_path = os.path.join(".", filename)
    HTML_file = open(file_path, "r", encoding='UTF-8')
    index = HTML_file.read()
    doc = BeautifulSoup(index, "html.parser")
    return doc

In [6]:
def generate_current_price(doc):
    goods_list = []
    
    prices = doc.find_all("span", {"class" : "s-item__price"})
    item_names = doc.find_all("h3", {"class" : "s-item__title"})[1:]
    subtitles = doc.find_all("div", {"class" : "s-item__subtitle"})
    
    for index in range (min(len(prices), len(item_names), len(subtitles))):
        product = [item_names[index].getText(), prices[index].getText(), subtitles[index].getText()]
        goods_list.append(product)

    df = pd.DataFrame(goods_list, columns = ['Item_Names', 'Prices', "Subtitles"])
    
    brand_new_df = df[df["Subtitles"] == "Brand New"]
    
    price_series = df["Prices"].apply(price_extract).apply(price_correction)

    return round(price_series.astype(float).mean(), 2)

In [7]:
def price_extract(price):
    return price[1:]

In [8]:
def price_correction(price):
    return price.replace(",", "")

In [9]:
def generate_new_report(brand, model_name, model_year, original_sticker_price, current_avg_price):
    percent_increase = round((current_avg_price - original_sticker_price) / original_sticker_price * 100, 2)
    percent_incr_str = str(percent_increase) + "%"
    
    final_stats_list.append([current_avg_price, original_sticker_price, \
        percent_incr_str, "eBay", model_name, model_year, brand])
    
    print(final_stats_list)

In [10]:
def generate_csv_file():
    dataframe = pd.DataFrame(final_stats_list, columns = ["current price", "original staicker price", \
        "% increase in price", "site", "model name", "model year", "brand"])
    dataframe.to_csv("AMD_cards_stats.csv")

In [11]:
while (flag != "q" and flag != "quit"):
    flag = "a"
    brand = input("What is the brand of the video card: ")
    model_name = input("What is the model name: ")
    model_year = input("What is the model year: ")
    original_sticker_price = float(input("What is the original sticker price of the video card: "))
    
    file_name = scrape_from_web(model_name)
    
    document = parse_HTML(file_name)
    
    current_avg_price = generate_current_price(document)
    
    generate_new_report(brand, model_name, model_year, original_sticker_price, current_avg_price)
    
    while (flag != "q" and flag != "c"):
        flag = input("Do you want to continue for next model: ")
        
generate_csv_file()

What is the brand of the video card: AMD
What is the model name: Radeon RX 6800
What is the model year: 2020
What is the original sticker price of the video card: 579
[[1189.0, 579.0, '105.35%', 'eBay', 'Radeon RX 6800', '2020', 'AMD']]
Do you want to continue for next model: c
What is the brand of the video card: AMD
What is the model name: Radeon RX 6700 XT
What is the model year: 2021
What is the original sticker price of the video card: 479
[[1189.0, 579.0, '105.35%', 'eBay', 'Radeon RX 6800', '2020', 'AMD'], [838.72, 479.0, '75.1%', 'eBay', 'Radeon RX 6700 XT', '2021', 'AMD']]
Do you want to continue for next model: c
What is the brand of the video card: AMD
What is the model name: Radeon RX 5600 XT
What is the model year: 2020
What is the original sticker price of the video card: 279
[[1189.0, 579.0, '105.35%', 'eBay', 'Radeon RX 6800', '2020', 'AMD'], [838.72, 479.0, '75.1%', 'eBay', 'Radeon RX 6700 XT', '2021', 'AMD'], [655.86, 279.0, '135.08%', 'eBay', 'Radeon RX 5600 XT', '20